In [ ]:
from zai import ZhipuAiClient
import json

# 初始化客户端（替换为你的 API Key）
client = ZhipuAiClient(api_key="2ae1442dccfd4c229f4e79add860c9a8.ETa0G51KArurOVVZ")

# 待抽取实体关系的英文文本（可替换为你的目标文本）
target_text = """Zhipu AI Open Platform is an artificial intelligence capability open platform launched by Zhipu HuaZhang Co., Ltd. 
The platform provides large language model services including GLM-4 and GLM-3, and its headquarters is located in Haidian District, Beijing."""

# 调用模型进行英文实体关系抽取
response = client.chat.completions.create(
    model="glm-4.7-flash",
    messages=[
        {
            "role": "system",
            "content": """You are an expert in English entity and relation extraction. Please follow these instructions:
1. Extract core entities from the text (e.g., organizations, products, locations, companies);
2. Extract semantic relations between entities (e.g., "developed by", "offers", "headquartered in");
3. Output ONLY a valid JSON object with the following structure:
{
    "entities": ["Entity1", "Entity2", ...],
    "relations": [
        {"subject": "EntityA", "relation": "relation_type", "object": "EntityB"},
        ...
    ]
}
4. Do NOT include any additional text, explanations, or markdown. Only pure JSON."""
        },
        {
            "role": "user",
            "content": f"Extract entities and relations from the following English text:\n\n{target_text}"
        }
    ],
    thinking={
        "type": "enabled"
    },
    max_tokens=4096,          # 英文通常不需要 65536，4096 足够
    temperature=0.1,
    response_format={"type": "json_object"}  # Enforce JSON output
)

# Parse and display results
try:
    result_text = response.choices[0].message.content
    result = json.loads(result_text)
    
    print("Extracted Entities:")
    for entity in result["entities"]:
        print(f"- {entity}")
    
    print("\nExtracted Relations:")
    for rel in result["relations"]:
        print(f"- {rel['subject']} → {rel['relation']} → {rel['object']}")
        
except json.JSONDecodeError:
    print("Model output is not valid JSON. Raw response:")
    print(response.choices[0].message.content)

: 

In [29]:
from sentence_transformers import SentenceTransformer
import numpy as np

# ==============================
# 替换为英文嵌入模型的本地路径
# ==============================
local_model_path = "/Users/xry/.cache/modelscope/hub/models/BAAI/bge-base-en-v1.5"

# ==============================
# 加载本地 SentenceTransformer 模型
# ==============================
print("正在加载英文嵌入模型...")
model = SentenceTransformer(local_model_path)

# ==============================
# 英文示例句子（必须是英文！）
# ==============================
sentences_1 = [
    "What is a large language model?",
    "How does photosynthesis work?"
]
sentences_2 = [
    "LLMs are trained on vast text corpora to generate human-like responses.",
    "Plants convert sunlight into chemical energy through photosynthesis."
]

# ==============================
# 生成归一化的嵌入向量
# ==============================
embeddings_1 = model.encode(sentences_1, normalize_embeddings=True)
embeddings_2 = model.encode(sentences_2, normalize_embeddings=True)

# ==============================
# 计算余弦相似度（归一化后即为点积）
# ==============================
similarity = embeddings_1 @ embeddings_2.T

# ==============================
# 输出结果
# ==============================
print("Similarity Matrix:")
print(np.round(similarity, 4))

正在加载英文嵌入模型...


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1454.75it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: /Users/xry/.cache/modelscope/hub/models/BAAI/bge-base-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Similarity Matrix:
[[0.6846 0.4103]
 [0.3698 0.7935]]


In [22]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# ==============================
# 替换为英文重排模型的本地路径
# ==============================
# 假设你已下载模型到此目录（可通过 Hugging Face 或 ModelScope 下载）
local_model_path = "/Users/xry/.cache/modelscope/hub/models/BAAI/bge-reranker-base"  # ← 改为 base 英文版

# ==============================
# 加载 tokenizer 和模型（强制 CPU + float32）
# ==============================
print("正在加载英文重排模型...")
tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForSequenceClassification.from_pretrained(
    local_model_path,
    torch_dtype=torch.float32,
    device_map="cpu"
)
model.eval()

# ==============================
# 构造英文查询和候选文本
# ==============================
query = "What is a panda?"
candidate_texts = [
    "Hello, how can I help you today?",               # irrelevant
    "The giant panda is a bear species endemic to China.",  # relevant
    "Pandas mainly live in Sichuan, Shaanxi, and Gansu provinces of China."  # highly relevant
]

# ==============================
# 批量计算重排得分
# ==============================
scores = []
with torch.no_grad():
    for text in candidate_texts:
        inputs = tokenizer(
            query,
            text,
            truncation=True,
            max_length=512,
            return_tensors="pt",
            padding=True
        ).to("cpu")  # 确保在 CPU 上

        outputs = model(**inputs)
        # 注意：bge-reranker 输出 logits 是 shape [1, 1]，取 [0][0]
        score = torch.sigmoid(outputs.logits[0][0]).item()
        scores.append(score)

# ==============================
# 按得分排序并输出
# ==============================
ranked_results = sorted(
    zip(candidate_texts, scores),
    key=lambda x: x[1],
    reverse=True
)

print("Reranking scores:", [round(s, 4) for s in scores])
print("\nReranked results (highest to lowest):")
for idx, (text, score) in enumerate(ranked_results, 1):
    print(f"Rank {idx} (score={score:.4f}): {text}")

正在加载英文重排模型...


Loading weights: 100%|██████████| 201/201 [00:00<00:00, 2152.25it/s, Materializing param=roberta.encoder.layer.11.output.dense.weight]              
XLMRobertaForSequenceClassification LOAD REPORT from: /Users/xry/.cache/modelscope/hub/models/BAAI/bge-reranker-base
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Reranking scores: [0.0, 0.5174, 0.0176]

Reranked results (highest to lowest):
Rank 1 (score=0.5174): The giant panda is a bear species endemic to China.
Rank 2 (score=0.0176): Pandas mainly live in Sichuan, Shaanxi, and Gansu provinces of China.
Rank 3 (score=0.0000): Hello, how can I help you today?
